# Data Pre-Processing

In [ ]:
# Imports
import pandas as pd
import datasets
from datasets import Dataset, DatasetDict

In [ ]:
# Imports for text preprocessing
import string
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
# nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

## YELP

Prep for RoBERTa-base

In [ ]:
# Imports
from transformers import pipeline
from transformers import AutoTokenizer

In [ ]:
ds = load_dataset('yelp_review_full')

ds # View

In [ ]:
# Inspect
ds['train'][0], ds['train'].features

{'label': 4,
 'text': "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."}

In [ ]:
ds_train = pd.DataFrame(ds['train'])
ds_test = pd.DataFrame(ds['test'])

display(ds_train.head()), display(ds_test.head())

In [ ]:
# assign the splits
train = Dataset.from_pandas(ds_train)
test = Dataset.from_pandas(ds_test)

# reconstruct both datasets into a Dataset Dict object
new_ds = DatasetDict(
    {
        'train': train,
        'test': test
    }
)
# view the resulting dataset dict object
new_ds

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

### EDA

In [ ]:
train.shape, test.shape

NameError: name 'train' is not defined

In [ ]:
display(train.head()), display(test.head())

### Clean, Tokenize

> Autotokenizer? Autoencoder?